### Configure your ArcGIS Online or Enterprise cnonnections, and specify a group from your src_gis to be copied.

In [61]:
import arcgis
from arcgis.gis import GIS
from arcgis.mapping import WebMap
import csv
from datetime import datetime
csv_file = r'C:/Users/bob10704/Desktop/Projects/1.Active Projects/HHS_HubMigration/hub_itemIDs.csv'

In [62]:
src_gis_url = 'https://hhsgov.maps.arcgis.com'     #https://grid3.maps.arcgis.com - can also use 'home' if you are running the notebook from the org you want to use as the src. 
src_group_id = 'a1e26ee36cb44ad2b97e0a57b9b1d6f8'
src_gis_username = 'HHS_PROTECT_PUBLIC' # Your username for the source organization
src_gis_password = 'nN96#wz@&EGP' # provide a password for that user, if None is left, you will be prompted to provide the password.
#src_client_id = 'aCa70VOCEMfck6ya' #appID for source org; OAuth

target_gis_url = 'https://ps-dbs.maps.arcgis.com'  #https://myneworg.maps.arcgis.coNm
target_gis_username = 'bhorne_PS_DBS' # Username for Target Org
target_gis_password = '78tHbDv5' # Password for Target Org
target_folder_name = 'HHS Hub Migration' #Name for the Folder in the target user's content that you want to copy content into.
target_group_ID = '5486511c3e614aa6b2f4b4d258d025d6'
#target_client_id = 'u4z280xNRnQn2JWU' #appID for target org; OAuth

src_gis = arcgis.gis.GIS(src_gis_url, src_gis_username, src_gis_password)
#src_gis = arcgis.gis.GIS(src_gis_url, client_id='aCa70VOCEMfck6ya')
target_gis = arcgis.gis.GIS(target_gis_url, target_gis_username, target_gis_password)
print("Successfully logged in as: " + src_gis.properties.user.username)
print("Successfully logged in as: " + target_gis.properties.user.username)

Successfully logged in as: HHS_PROTECT_PUBLIC
Successfully logged in as: bhorne_PS_DBS


#### Validate how many items are in the group to be copied.

In [63]:
src_group = src_gis.groups.get(src_group_id)
src_group_content = src_group.content()
print("{} Total items in Group to be copied".format(len(src_group_content)))

73 Total items in Group to be copied


In [68]:
content_group= 'HHS Hub Migration Content'
group_search = f'title:content_group'
map_group= target_gis.groups.search(group_search, max_groups=10)
print(map_group)

[]


#### Execute the copy! This code below also accounts for hosted tile layers, navigating the item relationship back to the source item, copying that item, and republishing it in the target system.

In [ ]:
for src_item in src_group_content:
    print("Copying Item: {} - {}".format(src_item.title, src_item.type))
    try:
        #Handle Hosted Tile Layers
        if 'Hosted Service' in src_item.typeKeywords and 'Map Service' in src_item.typeKeywords:
            if len(target_gis.content.search("source-{}".format(src_item.id)))>0:
                print("\t Item Already copied")
                continue
            #Clone TPK:
            print("\t Hosted Tile Layer found, copying source tile package")
            tpk_result = target_gis.content.clone_items(src_item.related_items('Service2Data'),folder=target_folder_name)
            if tpk_result == []:
                tpk_result = target_gis.content.search("source-{}".format(src_item.related_items('Service2Data')[0].id))
            print("\t Source Tile Package copied as {}".format(tpk_result[0].id))
            print("\t Publishing service...")
            service_result = tpk_result[0].publish(build_initial_cache=True)
            service_result.update({"typeKeywords":service_result.typeKeywords+["source-{}".format(src_item.id)]})
            print("\t Service Result: {}".format(service_result))
        else:
            result = target_gis.content.clone_items([src_item],group=target_folder_name)
            if result == []:
                print("\t Item Already copied")
            else:
                print("\t Result: {}".format(result))
   
    except Exception as e:
        if "already exists" in str(e):
            print("\tService already copied to target organization")
        else:
            print("\t Error copying item: {} -- {} ".format(src_item.title, e))
            


In [44]:
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")

In [55]:
mygroup = target_gis.groups.search('5486511c3e614aa6b2f4b4d258d025d6')
migration = mygroup[0]
print(mygroup)

[<Group title:"HHS Hub Migration  Content" owner:bhorne_PS_DBS>]


In [58]:
migration_content = mygroup[0].content()

In [60]:
with open(csv_file.format(dt_string), 'w') as csvfile:
    dataWriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    dataWriter.writerow(['Title',
                         'Item ID',
                         'Item Type'])
    for item in migration_content:
        dataWriter.writerow([
            item.title,
            item.id,
            item.type
        ])

2a456d96d79a4f6ebdf1ebe4eb227b16
cc37eab2e81749a5a462c4a9e4180086
3ee6bfb9c9034738892fd28f6d24c6ba
a41bed03211647f4adc9aab2a07f72cd
80374b43e9c848cc924e28cddb6993bc
60e2e59e0eb344c29c63aaf4aedda12b
0f1f450f78ff4c3dbe1619c73c053c1f
b7876e769f0242b99e7579e7df957d7d
3a397f93ec904d1ba4e0181e2def2db1
d953680d5f99478e9d33a3a1d900d7ee
3afb0464379140a5a6732aa84bc16339
7350c3af539448e3ba107e623021015b
32d35878f7d3498d9b55f1dbf8aceba5
3c781ae10d4b45e99e327f6af7a8dc9e
1442a1b41531486688c668afde1beb0a
143b9a468414469dacd9d49b4e7c60e0
ee44df5d3d1c4e7e9a62db871fcb7125
a83feaf0a69b49dba78b202701e026f7
e513756f5bd04ac0b8c45857263a91f4
00bedbac15824fb7a7e566e2480740eb
